In [1]:
! pip install semantic-kernel -U

In [2]:
import os
import sys

import semantic_kernel as sk

In [3]:

kernel = sk.Kernel()

In [4]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [5]:
service_id = None

In [6]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_service(skaoai.AzureChatCompletion(deployment_name=deployment,endpoint=endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [7]:
from semantic_kernel.planners.basic_planner import BasicPlanner

In [8]:

planner = BasicPlanner(service_id)

In [9]:
base_skills_directory = '../../../plugins'

In [10]:
import APIPlugin.CustomAPIPlugin as custom_plugin

In [11]:

custom_plugin = kernel.import_plugin_from_object(custom_plugin.CustomAPIPlugin(), plugin_name="CustomAPI")  #.import_native_skill_from_directory(base_plugin , "APIPlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "TranslatePlugin")

In [12]:
ask = """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate email to Chinese"""

In [13]:
service_id

In [14]:

plan = await planner.create_plan(ask, kernel)

In [15]:
print(plan.generated_plan)

{
    "input": "Travelling Tips",
    "subtasks": [
        {"function": "CustomAPI.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips", "args": {"input": "weather"}},
        {"function": "EmailPlugin.WeatherMail", "args": {"input": "weather tips"}},
        {"function": "TranslatePlugin.MultiLanguage", "args": {"language": "Chinese"}}
    ]
}


In [16]:
results = await planner.execute_plan(plan, kernel)

In [17]:
results

'主题：重要的天气和穿着提示，以确保安全出行\n\n亲爱的团队，\n\n希望这封邮件找到你们时一切都好。我们都知道，我们的工作经常需要我们去不同的地方，而各地的天气条件可能会有很大的差异。因此，及时了解天气预报并适当穿着是至关重要的，以确保我们的舒适和安全。\n\n以下是我们应该牢记的一些天气提示：\n\n1. 出发前总是要检查你的目的地的天气预报。这将帮助你适当地打包行李并规划你的活动。\n\n2. 注意严重的天气警告。如果有任何关于暴风雨、大雨或极端温度的警报，可能更安全的做法是重新安排你的行程。\n\n3. 如果你要去的地方气候炎热，记得要保持水分并使用防晒霜来保护你的皮肤免受有害紫外线的伤害。\n\n4. 对于寒冷的气候，穿多层衣服以保暖。别忘了带帽子、手套和一件暖和的大衣。\n\n5. 如果你要去的地方有很大的下雨概率，记得要带一件防水夹克和雨伞。\n\n请记住，成功的旅行的关键是准备。适应天气的穿着不仅可以确保你的舒适，还可以降低患上中暑或低体温症等健康问题的风险。\n\n请始终注意旅行条件。你们的安全和健康是我们的首要任务。如果你有任何疑虑或需要更多的信息，随时联系我们。\n\n祝你们旅途平安！\n\n顺祝商祺，\n\n[你的名字]\n\n[你的职位]'